In [3]:
# from google.colab import drive

# drive.mount('/content/drive')

# Run the following 2 blocks. 
## Then, start where it says Step 1/7. Run all blocks unless specified otherwise.

In [27]:
import numpy as np
import struct
import array

import ipywidgets as widgets
import matplotlib.pyplot as plt
import sys
import re
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

from lmfit import Model

import sys
import tkinter
from tkinter.filedialog import askopenfilenames

The code block below is modified by Tiger Mou from the Python3convert-multipak-to-XPSPEAK-columbia.py file that was originally written by Robert Forest in Python 2, modified for Python 3 by Hansen Mou. Plotting and fitting was written by Hansen Mou.

In [28]:
### This block imports the spe file.

class XSpectrum:

    def __init__(self):
        self._regions = []
        self._XPS_REGION_START_STR = b'DP'
        self._XPS_HEADER_SIZE = 42
   
    def load_xps(self, path):
        """
        Read and parse XPS file according to the reverse engineering of
        Robert Forest's Python3convert-multipak-to-XPSPEAK-columbia.py
        """
        file_str = ""
        # We read in the entire file because iterating through is tedious.
        with open(path, 'rb') as f:
            print('Reading XPS from ' + path)
            file_str = f.read();
            f.close()

        version_str = file_str[0:11].decode('utf-8')
        print(f'File version is: {version_str}')

        idx = file_str.find(self._XPS_REGION_START_STR) + len(self._XPS_REGION_START_STR)
        while idx != -1:
            print(f'IDX: {idx}')
            (region, size) = self.__xps_read_region(file_str, idx)
            self._regions.append(region)
            idx = file_str.find(self._XPS_REGION_START_STR, idx + size) + len(self._XPS_REGION_START_STR)

        print('Done!')
    
  
    def load_spe(self, path):
        """
        Read and parse SPE file according to the reverse engineering of
        Robert Forest's Python3convert-multipak-to-XPSPEAK-columbia.py
        """
        with open(path, 'rb') as f:
            print('Reading SPE from ' + path)
            num_regions = self.__spe_get_number_of_regions(f)
            BE_parameters = self.__spe_get_BE_parameters(f, num_regions)
            self.__spe_convert_multipak_file(f, BE_parameters, num_regions)
        print('Done!')

    # XPS helper functions
    def __xps_read_region(self, file_str, idx):
        """
        """
        curr_reg = dict()

        """ Header contains: (total 42 bytes)
        0: region_size (2 bytes)
        1: unknown: uint 655353 (4)
        2: 20 characters for the region_name, with whitespace padding (20)
        3: region_size again ?? (2)
        4: unknown: int of 1 (2)
        5: region_size + 1 (2)
        and 10 zeros to end the header (10)
        """
        region_header = struct.unpack_from('<HI20sHHH10x', file_str, idx)
        print(region_header)
        idx += self._XPS_HEADER_SIZE

        curr_reg['region_size'] = region_header[0]
        curr_reg['region_name'] = region_header[2].strip()
        
        # Read region be
        curr_reg['be_data'] = np.array(struct.unpack_from(f'<{region_header[0]}f', file_str, idx))
        idx += region_header[0] * 4

        # Strange whitespace of 1, region_size+1, and 10 zeros
        unknown_whitespace = struct.unpack_from('<HH10x', file_str, idx)
        idx += 14

        # Read region intensity
        curr_reg['intensity'] = np.array(struct.unpack_from(f'<{region_header[0]}f', file_str, idx))
        idx += region_header[0] * 4

        """ Footer contains:
        0: max_BE
        1: min_BE
        2: max_intensity
        3: min_intensity
        and LOTS of whitespace for some reason...
        """
        region_footer = struct.unpack_from('<4f', file_str, idx)
        idx += 76

        curr_reg['max_be'] = region_footer[0]
        curr_reg['min_be'] = region_footer[1]
        curr_reg['max_intensity'] = region_footer[2]
        curr_reg['min_intensity'] = region_footer[3]

        # I think it's nothing but idk why
        region_post_footers = []
        for i in range(6):
            region_post_footers.append(struct.unpack_from(f'<HH10x{region_header[0]}f', file_str, idx + i*18+region_header[0]*4))

        curr_reg['post_footers'] = region_post_footers
        
        region_size = self._XPS_HEADER_SIZE + region_header[0] * 4 * 2 + 14 + 16 + (6 *(2+2+region_header[0]*4+10)) + 18

        return (curr_reg, region_size)


    # SPE helper functions

    def __spe_get_number_of_regions(self, file):
        """
        Returns the total number of elements ("regions") from multi-scan
        """
        file_string = file.read()
        i = file_string.rfind(b'NoSpectralReg: ') + len('NoSpectralReg: ')
        file.seek(i)
        return int(file.readline())

    def __spe_get_BE_parameters(self, file, num_regions):
        """Returns a 2D list of binding energy parameters as strings
        First dimension is the region number
        Second dimension is a list of individual parameters for that region
        Indices: 3-region name, 5-region size, 6-step size, 7-high BE, 8-low BE
        """
        BE_parameters = []
        for line_number in range(num_regions):
            BE_parameters.append(file.readline())
            BE_parameters[line_number] = BE_parameters[line_number].split(b' ')
        return BE_parameters

    def __spe_convert_multipak_file(self, file, BE_parameters, num_regions):
        # Set up regions.
        self._regions = [dict() for _ in range(num_regions)]

        # Go to start of data in multipak file.
        data_size = 0
        for i in range(num_regions):
            data_size += int(BE_parameters[i][5])
        file.seek(-8 * data_size, 2) 

        # Process data in each region.
        for i in range(num_regions):
            # Reference to regions dict.
            curr_reg = self._regions[i]

            region_params = BE_parameters[i]
            curr_reg['max_be'] = float(region_params[7])
            curr_reg['min_be'] = float(region_params[8])
            curr_reg['region_size'] = int(region_params[5])
            intensity_data_double = array.array('d')
            intensity_data_double.fromfile(file, curr_reg['region_size'])
                        
            # Region metadata.  Not sure if useful.
            curr_reg['region_name'] = region_params[3]

            # I guess XPS format doesn't support double?
            curr_reg['be_data'] = np.linspace(curr_reg['max_be'], curr_reg['min_be'], curr_reg['region_size'])

            curr_reg['intensity'] = np.array(intensity_data_double)

            curr_reg['min_intensity'] = min(curr_reg['intensity'])
            curr_reg['max_intensity'] = max(curr_reg['intensity'])

def get_file_names():    
    root = tkinter.Tk()
    root.withdraw()
    root.attributes('-topmost',True)
    file_list = askopenfilenames(parent=root, title='Open XPS files',
                                              filetypes=[('.spe', '*.spe')])
    file_list = root.tk.splitlist(file_list) #workaround for Windows bug
    root.destroy()
    return file_list

## Step 1/7: Start here! Run the block below to import your SPE file and get a plot of the raw data.
You can mouse over the plot to see the coordinates!

In [29]:
### Import file
file = get_file_names()

data1 = XSpectrum()

if len(file) > 0:
    data1.load_spe(file[0])

### Identify the regions in the imported file
reg_len = len(data1._regions)
label = []
for i in range(reg_len):
    label.append(data1._regions[i]['region_name'].decode('UTF-8'))
print("\nRegions identified: " + str(label))

### Assigning names to each set of data
id_data1 = {}
for i in range(reg_len):
    id_data1[label[i]] = data1._regions[i]


### Plot the raw data
%matplotlib widget
sub_tab=[widgets.Output() for i in range(reg_len)]
tab = widgets.Tab(sub_tab)
for i in range(reg_len):
    tab.set_title(i,label[i].format(i+1))
    with sub_tab[i]:
        fig = plt.figure(figsize=(10,5))
        ax = fig.subplots()
        ax.plot(data1._regions[i]['be_data'], data1._regions[i]['intensity'])
        ax.set_title(label[i],fontweight='bold',fontsize=20)
        ax.invert_xaxis()
        ax.set_xlabel('Binding Energy [eV]',fontweight='bold',fontsize=16)
        ax.set_ylabel('Intensity',fontweight='bold',fontsize=16)
        ax.tick_params(axis='both',which='major',labelsize=14)
        ax.xaxis.set_minor_locator(MultipleLocator(100))
        plt.show(fig)
display(tab)
def mouse_move(event):
    x, y = event.xdata, event.ydata
    print(x, y)
plt.connect('motion_notify_event', mouse_move)
plt.show()

Reading SPE from C:/Users/hm115/Google Drive/Forschung/Data/XPS/XPS Data/NbC-film/210630-NbC-film-16'1A-15+58min2_fine0003.SPE
Done!

Regions identified: ['Nb3d', 'Pt4f7', 'C1s', 'O1s']


## Step 2/7: Choose the region you want to fit

In [30]:
which_reg = "Nb3d"

if which_reg in id_data1:
    be_vals = id_data1[which_reg]['be_data']
    activ_reg = label.index(which_reg)
    print(str(which_reg) + " loaded!")
else:
    print("The region you chose is not present. Please choose from the following:")
    for i in id_data1:
        print("\t"+str(i))

Nb3d loaded!


## Step 3/7: Set the upper and lower bounds for the background

In [31]:
### Setting background

background_lower_bound = 206.13
background_upper_bound = 216.03

x_min = be_vals[np.abs(be_vals - background_lower_bound).argmin()]
x_max = be_vals[np.abs(be_vals - background_upper_bound).argmin()]
if x_min > x_max:
    print("Please switch your upper and lower bounds.")
elif x_max not in be_vals or x_min not in be_vals:
    print("Your selected bounds exceed the region. Please set bounds between " + str(be_vals[-1]) + " and " + str(be_vals[0]))
else:
    ind_min = np.where(be_vals == x_min)[0][0]
    ind_max = np.where(be_vals == x_max)[0][0]

    intensity_vals = id_data1[which_reg]['intensity']
    y_min = id_data1[which_reg]['intensity'][ind_min]
    y_max = id_data1[which_reg]['intensity'][ind_max]

    print("Done!")

Done!


### Implementing the Shirley Background Algorithm
(Should rewrite this into a function)

In [32]:
### Shirley background algorithm

### Brief explanation of the algorithm: Shirley background is generated to be S(E) = y_min + interval * { A(2) / [A(1)+A(2)] }
##  Where S(E) is the background intensity at a given binding energy E
##  A(2) is the area at binding energies less than E, and A(1) is area at binding energies greater than E
##  interval is usually accepted to be the intensity difference of the lower and upper bounds for binding energy, for our background region of interest
##  This implementation first calculates the total area [A(1) + A(2)] by numerically integrating via the trapezoid method to find k_integral, and then k_val is simply " interval / [A(1) + A(2)]"
##  Next we calculate A(2) (designated y_integral) and multiply it with k_val
##  Once the iteration has completed, we add y_min to the background
##  Tolerance is determined by calculating the norm of the difference of the initial and final background arrays. I don't understand how that works, but norm basically indicates the degree of difference between 2 arrays, so the smaller the norm, the higher tolerance. Unit is arbitrary, I think.

shirley_back = np.zeros(be_vals.shape)      ## initialize shirley_background array, same size as the entire domain of the region
for i in range(ind_min,ind_max+1):          ## populate shirley_back array with values for linear background
    shirley_back[i] = ((y_max - y_min)/(x_max - x_min)) * (be_vals[i] - x_min)
shirley_back_it = shirley_back.copy()       ## copy shirley_back array for a separate, iterated/manipulated version

tol = 0.01      ## Tolerance level

it_max = 1000       ## maximum number of iterations
it = 0              ## initial iteration value

while it < it_max:
    k_integral = 0.0
    for i in range(ind_max, ind_min):
        k_integral += (be_vals[i+1] - be_vals[i]) * 0.5 * (intensity_vals[i+1] + intensity_vals[i] - 2 * y_min - shirley_back[i+1] - shirley_back[i]) # integration via trapezoid method
    k_val = (y_max - y_min)/k_integral
        
    for i in range(ind_max, ind_min):
        y_integral = 0.0
        for j in range(i, ind_min):
            y_integral += (be_vals[j+1] - be_vals[j]) * 0.5 * (intensity_vals[j+1] + intensity_vals[j] - 2 * y_min - shirley_back[j+1] - shirley_back[j])
        shirley_back_it[i] = k_val * y_integral
    
    if np.linalg.norm(shirley_back_it - shirley_back) < tol:
        shirley_back = shirley_back_it.copy()
        break
    else:
        shirley_back = shirley_back_it.copy()
    it +=1

if it >= it_max:
    print("max iterations exceeded")

shirley_back_fin = y_min + shirley_back

print("Number of iterations: " + str(it))
# print(shirley_back_fin)

### Replotting the region with the Shirley background
%matplotlib widget
fig = plt.figure(figsize=(10,5))

i = activ_reg

ax = fig.add_subplot(111)
ax.plot(be_vals, intensity_vals)       ## Plot Region in question
ax.plot(be_vals[ind_max:ind_min],shirley_back_fin[ind_max:ind_min])    ## Plot Shirley background
ax.set_title(label[i],fontweight='bold',fontsize=20)
ax.invert_xaxis()
plt.legend(['data', 'background'])
ax.set_xlabel('Binding Energy [eV]',fontweight='bold',fontsize=16)
ax.set_ylabel('Intensity',fontweight='bold',fontsize=16)
ax.tick_params(axis='both',which='major',labelsize=14)
fig.subplots_adjust(hspace = 0.4)
ax.xaxis.set_minor_locator(MultipleLocator(100))
def mouse_move(event):
    x, y = event.xdata, event.ydata
    print(x, y)
plt.connect('motion_notify_event', mouse_move)
plt.show()

Number of iterations: 5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Todo: incorporate peak area into fit model, fit paired dependent peaks, make process more user friendly

### Defining the basic fit equatons (Gaussian, Lorentzian)

In [33]:
### Gaussian
def gauss(x,E,F,area,m):
    # E = pars['E']
    # F = pars['F']
    # # a = param[2]
    # m = pars['m']
    # model = height* np.exp(-4 * np.log(2) * (1 - m/100) * ((x - E) / F)**2)
    model = (2 * np.sqrt(np.log(2))/(F * np.sqrt(np.pi))) * area * np.exp(-4 * np.log(2) * (1 - m/100) * ((x - E) / F)**2)
    return model

# Note: for Gaussian, FWHM = 2 * sigma * sqrt(2 ln(2))

### Lorentzian
def lorentz(x,E,F,area,m):
    # E = pars['E']
    # F = pars['F']
    # # a = param[2]
    # m = pars['m']
    # model = height / ((1 + 4 * m/100 * ((x - E) / F)**2))
    model = 2 * area / (np.pi * (1 + 4 * m/100 * ((x - E) / F)**2) * F)
    return model
# !!!! I don't understand why this equation shouldn't be multiplied by 2. !!!!

# Note: for Lorentzian, FWHM = 2 sigma

### E = position (eV); F = FWHM; m = % Lorentzian

### Sum G/L
def sum_gl(x,E,F,area,m):
    # E = param[0]
    # F = param[1]
    # a = param[2]
    # m = param[3]
    model = ((1-m/100) * gauss(x,E,F,area,0) + (m/100) * lorentz(x,E,F,area,100))
    return model

### Product G/L
## Has issues with returning the correct area. I think it has to do with how the weight factor 'm' is incorporated. May need to do numerical integration.
def product_gl(x,E,F,area,m):
    # E = pars['E']
    # F = pars['F']
    # # a = param[2]
    # m = pars['m']
    model = (gauss(x,E,F,area,m) * lorentz(x,E,F,area,m))
    return model

In [36]:
i = activ_reg

intensity_zeroed = np.zeros(be_vals.shape)

for j in range(len(be_vals)):
    intensity_zeroed[j] = intensity_vals[j] - shirley_back_fin[j]

norm_factor = np.max(intensity_zeroed)

intensity_norm = intensity_zeroed / norm_factor
#print(intensity_norm)

# print(len(intensity_zeroed))
# print(norm_factor)

# line_fit_zeroed = product_gl(data1._regions[i]['be_data'][ind_max:ind_min],80,param)
# line_fit_zeroed = product_gl(be_vals,param)
# line_fit_based = np.zeros(be_vals.shape)

# for j in range(len(shirley_back_fin)):
#     line_fit_based[j] = line_fit_zeroed[j] + shirley_back_fin[j]

## Step 4/7: Define your peaks and set initial Parameters
### Setting the Fit Parameters

In [40]:
### param = [peak binding energy, FWHM, area, % gauss/lorentz]
param1 = [210.5,1.0,300,80]
param2 = [213.7,1.0,200,80]
param3 = [208.8,1.0,100,80]
param4 = [211,1.0,80,80]

In [46]:
### Plots the initial guess on the figure

# This isn't automatically scaled yet. You need to add each one manually if you want to see the initial guess.
initial_fit_1 = sum_gl(be_vals[ind_max:ind_min],*param1) + shirley_back_fin[ind_max:ind_min]
initial_fit_2 = sum_gl(be_vals[ind_max:ind_min],*param2) + shirley_back_fin[ind_max:ind_min]
initial_fit_3 = sum_gl(be_vals[ind_max:ind_min],*param3) + shirley_back_fin[ind_max:ind_min]
initial_fit_4 = sum_gl(be_vals[ind_max:ind_min],*param4) + shirley_back_fin[ind_max:ind_min]

# make sure that you multiply the shirley_back_fin by (total number of initial fits minus 1!)
initial_fit_sum = initial_fit_1 + initial_fit_2 + initial_fit_3 + initial_fit_4 - 3 * shirley_back_fin[ind_max:ind_min]

fig = plt.figure(figsize=(10,5))

ax = fig.add_subplot(111)


ax.plot(be_vals, intensity_vals)       ## Plot Region in question
ax.plot(be_vals[ind_max:ind_min],shirley_back_fin[ind_max:ind_min])    ## Plot Shirley background
ax.plot(be_vals[ind_max:ind_min],initial_fit_sum)        ## Initial Fit
ax.plot(be_vals[ind_max:ind_min],initial_fit_1)        ## Initial Fit
ax.plot(be_vals[ind_max:ind_min],initial_fit_2)        ## Initial Fit



ax.set_title(label[activ_reg],fontweight='bold',fontsize=20)
ax.invert_xaxis()
ax.set_xlabel('Binding Energy [eV]',fontweight='bold',fontsize=16)
ax.set_ylabel('Intensity',fontweight='bold',fontsize=16)
ax.tick_params(axis='both',which='major',labelsize=14)
fig.subplots_adjust(hspace = 0.4)

# plt.legend(['data', 'background', 'initial guess', 'final fit'])
plt.legend(['data', 'background', 'overall fit'])

ax.xaxis.set_minor_locator(MultipleLocator(100))

# ax[0].set_ylim(400,480)
# ax[1].set_ylim(0,500)

def mouse_move(event):
    x, y = event.xdata, event.ydata
    print(x, y)

plt.connect('motion_notify_event', mouse_move)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Step 5.1/7: Run this block if fitting individual, unbound peaks. If not, do not run this block!!

In [453]:
peak_id = ['p1_','p2_']

peak1 = Model(sum_gl,prefix=peak_id[0])
# peak1 = Model(product_gl,prefix=peak_id[0])
pars = peak1.make_params()

pars['p1_E'].set(value=param1[0])
pars['p1_F'].set(value=param1[1],min=0,max=1.7)
pars['p1_area'].set(value=param1[2],min=0.1)
pars['p1_m'].set(value=param1[3],min=0,max=100)

# pars['p1_m'].vary = False

peak2 = Model(sum_gl,prefix=peak_id[1])
# peak2 = Model(product_gl,prefix=peak_id[1])
pars.update(peak2.make_params())

pars['p2_E'].set(value=param2[0])
pars['p2_F'].set(value=param2[1],min=0,max=1.7)
pars['p2_area'].set(value=param2[2],min=0.1)
pars['p2_m'].set(value=param2[3],min=0,max=100)

# pars['p2_m'].vary = False

print(pars)

model = peak1 + peak2

Parameters([('p1_E', <Parameter 'p1_E', value=534, bounds=[-inf:inf]>), ('p1_F', <Parameter 'p1_F', value=1.0, bounds=[0:1.7]>), ('p1_area', <Parameter 'p1_area', value=300, bounds=[0.1:inf]>), ('p1_m', <Parameter 'p1_m', value=80, bounds=[0:100]>), ('p2_E', <Parameter 'p2_E', value=536, bounds=[-inf:inf]>), ('p2_F', <Parameter 'p2_F', value=1.0, bounds=[0:1.7]>), ('p2_area', <Parameter 'p2_area', value=800, bounds=[0.1:inf]>), ('p2_m', <Parameter 'p2_m', value=80, bounds=[0:100]>)])


## Step 5.2/7: Run this block if fitting couplets.

In [54]:
### Main peak is 'p1_', bound couplet is 'p1_2'

peak_id = ['p1_','p1_2_','p2_','p2_2_']

peak1 = Model(sum_gl,prefix=peak_id[0])
# peak1 = Model(product_gl,prefix=peak_id[0])       ## product_gl doesn't provide accurate areas yet. Just use sum_gl
pars = peak1.make_params()

# parameters pulled from the list param1
pars['p1_E'].set(value=param1[0])
pars['p1_F'].set(value=param1[1],min=0,max=1.7)
pars['p1_area'].set(value=param1[2],min=0.1)
pars['p1_m'].set(value=param1[3],min=0,max=100)

peak2 = Model(sum_gl,prefix=peak_id[1])
# peak2 = Model(product_gl,prefix=peak_id[1])
pars.update(peak2.make_params())

### Set how the parameters for peak1_2 are bound by peak 1

# peak binding energy
pars['p1_2_E'].set(expr='p1_E + 2.72')

# peak FWHM
pars['p1_2_F'].set(expr='p1_F')

# peak area (could probably automate this by extracting the orbital from the region name)
pars['p1_2_area'].set(expr='p1_area * (2/3)')

# % GL
pars['p1_2_m'].set(expr='p1_m')
##########################################

### Couplet 2
peak3 = Model(sum_gl,prefix=peak_id[2])
# peak1 = Model(product_gl,prefix=peak_id[0])       ## product_gl doesn't provide accurate areas yet. Just use sum_gl
pars.update(peak3.make_params())

# parameters pulled from the list param1
pars['p2_E'].set(value=param3[0])
pars['p2_F'].set(value=param3[1],min=0,max=1.7)
pars['p2_area'].set(value=param3[2],min=0.1)
pars['p2_m'].set(value=param3[3],min=0,max=100)


peak4 = Model(sum_gl,prefix=peak_id[3])
pars.update(peak4.make_params())

### Set how the parameters for peak2_2 are bound by peak 2

# peak binding energy
pars['p2_2_E'].set(expr='p2_E + 2.72')

# peak FWHM
pars['p2_2_F'].set(expr='p2_F')

# peak area (could probably automate this by extracting the orbital from the region name)
pars['p2_2_area'].set(expr='p2_area * (2/3)')

# % GL
pars['p2_2_m'].set(expr='p2_m')

print(pars)

model = peak1 + peak2 + peak3 + peak4

Parameters([('p1_E', <Parameter 'p1_E', value=210.5, bounds=[-inf:inf]>), ('p1_F', <Parameter 'p1_F', value=1.0, bounds=[0:1.7]>), ('p1_area', <Parameter 'p1_area', value=300, bounds=[0.1:inf]>), ('p1_m', <Parameter 'p1_m', value=80, bounds=[0:100]>), ('p1_2_E', <Parameter 'p1_2_E', value=213.22, bounds=[-inf:inf], expr='p1_E + 2.72'>), ('p1_2_F', <Parameter 'p1_2_F', value=1.0, bounds=[-inf:inf], expr='p1_F'>), ('p1_2_area', <Parameter 'p1_2_area', value=200.0, bounds=[-inf:inf], expr='p1_area * (2/3)'>), ('p1_2_m', <Parameter 'p1_2_m', value=80, bounds=[-inf:inf], expr='p1_m'>), ('p2_E', <Parameter 'p2_E', value=208.8, bounds=[-inf:inf]>), ('p2_F', <Parameter 'p2_F', value=1.0, bounds=[0:1.7]>), ('p2_area', <Parameter 'p2_area', value=100, bounds=[0.1:inf]>), ('p2_m', <Parameter 'p2_m', value=80, bounds=[0:100]>), ('p2_2_E', <Parameter 'p2_2_E', value=211.52, bounds=[-inf:inf], expr='p2_E + 2.72'>), ('p2_2_F', <Parameter 'p2_2_F', value=1.0, bounds=[-inf:inf], expr='p2_F'>), ('p2_2_a

## Step 6/7: Run this block to perform the fit.
### Check the "[[Variables]]" block for the fitted parameters

In [58]:
fit = model.fit(intensity_zeroed[ind_max:ind_min],pars,x=be_vals[ind_max:ind_min],method="Least_squares")

print(fit.fit_report())

final = (fit.best_fit) + shirley_back_fin[ind_max:ind_min]
comps = fit.eval_components(x=be_vals[ind_max:ind_min])

for i in peak_id:
    comps[i] = comps[i] + shirley_back_fin[ind_max:ind_min]


[[Model]]
    (((Model(sum_gl, prefix='p1_') + Model(sum_gl, prefix='p1_2_')) + Model(sum_gl, prefix='p2_')) + Model(sum_gl, prefix='p2_2_'))
[[Fit Statistics]]
    # fitting method   = least_squares
    # function evals   = 13
    # data points      = 198
    # variables        = 8
    chi-square         = 79464.1277
    reduced chi-square = 418.232251
    Akaike info crit   = 1202.96920
    Bayesian info crit = 1229.27534
[[Variables]]
    p1_E:       211.670650 +/- 0.01096415 (0.01%) (init = 210.5)
    p1_F:       1.47626972 +/- 0.03415589 (2.31%) (init = 1)
    p1_area:    514.463173 +/- 19.8632172 (3.86%) (init = 300)
    p1_m:       1.8663e-12 +/- 14.7429804 (789972774234380.25%) (init = 80)
    p1_2_E:     214.390650 +/- 0.01096415 (0.01%) == 'p1_E + 2.72'
    p1_2_F:     1.47626972 +/- 0.03415589 (2.31%) == 'p1_F'
    p1_2_area:  342.975449 +/- 13.2421448 (3.86%) == 'p1_area * (2/3)'
    p1_2_m:     1.8663e-12 +/- 14.7429804 (789972775513084.25%) == 'p1_m'
    p2_E:       207.9

## Step 7/7: Run this block to plot the fit. You may have to change the labels and such.

In [57]:
fig = plt.figure(figsize=(10,5))

ax = fig.add_subplot(111)


ax.plot(be_vals, intensity_vals)       ## Plot Region in question
ax.plot(be_vals[ind_max:ind_min],shirley_back_fin[ind_max:ind_min])    ## Plot Shirley background

ax.plot(be_vals[ind_max:ind_min], final)    ## Final Fit

ax.plot(be_vals[ind_max:ind_min],comps['p1_'], label='p1')
# ax.plot(be_vals[ind_max:ind_min],comps['p2_'], label='p2')        # left this behind for 2nd individual peak
ax.plot(be_vals[ind_max:ind_min],comps['p1_2_'], label='p1_2')      # left this behind for 1_2 couplet peak
ax.plot(be_vals[ind_max:ind_min],comps['p2_'], label='p2')
ax.plot(be_vals[ind_max:ind_min],comps['p2_2_'], label='p2_2')


ax.set_title(label[activ_reg],fontweight='bold',fontsize=20)
ax.invert_xaxis()
ax.set_xlabel('Binding Energy [eV]',fontweight='bold',fontsize=16)
ax.set_ylabel('Intensity',fontweight='bold',fontsize=16)
ax.tick_params(axis='both',which='major',labelsize=14)
fig.subplots_adjust(hspace = 0.4)

plt.legend(['data', 'background', 'final fit'])

ax.xaxis.set_minor_locator(MultipleLocator(100))

def mouse_move(event):
    x, y = event.xdata, event.ydata
    print(x, y)

plt.connect('motion_notify_event', mouse_move)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

You're done! I haven't implemented a way to export the data yet, so just copy the plot and the results into powerpoint or something.

Please email me at h.mou@columbia.edu or on Slack with any questions or bugs to report.

Known issues:
- using product_gl method doesn't provide accurate areas
- current code doesn't easily scale to easily add more peaks
- not user friendly at all
- no way to easily export the data

Right now I have to zero & normalize the raw data, because the fit equations max out at 1 and start at 0 intensity, before fitting. Then I undo everything to move the fitted plot onto the original raw data.

## Ignore the cell below for now. It is simply the normalized & zeroed version of the plot.

In [ ]:
# %matplotlib widget
# fig = plt.figure(figsize=(10,5))

# ax = fig.add_subplot(111)

# ax.plot(be_vals, intensity_norm)       ## Plot Region in question
# ax.plot(be_vals[ind_max:ind_min],shirley_back[ind_max:ind_min]/norm_factor)    ## Plot Shirley background
# ax.plot(be_vals,line_fit_zeroed)

# ax.set_title(label[i],fontweight='bold',fontsize=20)
# ax.invert_xaxis()
# ax.set_xlabel('Binding Energy [eV]',fontweight='bold',fontsize=16)
# ax.set_ylabel('Intensity',fontweight='bold',fontsize=16)
# ax.tick_params(axis='both',which='major',labelsize=14)
# fig.subplots_adjust(hspace = 0.4)

# ax.xaxis.set_minor_locator(MultipleLocator(100))

# # ax[0].set_ylim(400,480)
# # ax[1].set_ylim(0,500)

# def mouse_move(event):
#     x, y = event.xdata, event.ydata
#     print(x, y)

# plt.connect('motion_notify_event', mouse_move)

# plt.show()